## LDA 토픽 모델링으로 콘텐츠 리뷰를 분석하자

https://velog.io/@mare-solis/LDA-%ED%86%A0%ED%94%BD-%EB%AA%A8%EB%8D%B8%EB%A7%81%EC%9C%BC%EB%A1%9C-%EC%BD%98%ED%85%90%EC%B8%A0-%EB%A6%AC%EB%B7%B0%EB%A5%BC-%EB%B6%84%EC%84%9D%ED%95%98%EC%9E%90

In [1]:
from konlpy.tag import Mecab
from tqdm import tqdm 
import re
import pickle
import csv
import pandas as pd
from pandas import DataFrame 
import numpy as np

#from konlpy.tag import Okt
#okt = Okt()

In [2]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

/Users/jamiejeong-eunpark/opt/anaconda3/envs/mecab/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
#data = pd.read_csv('comment_product_mapped.csv', low_memory=False)
data = pd.read_csv('data/과일견과쌀.csv', low_memory=False)
#정육계란, 채소, 수산해산, 과일견과쌀 - 81만

In [4]:
# 공백 삭제
data['리뷰 내용'] = data['리뷰 내용'].str.replace(r'\s+', '', regex=True)

# 정규표현식으로 한글이 아닌 문자 찾기
non_korean_pattern = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣]+')
data['리뷰 내용'] = data['리뷰 내용'].astype(str)
data['exclamation_mark'] = data['리뷰 내용'].apply(lambda x: non_korean_pattern.findall(x))

In [5]:
data['exclamation_mark']

0                       [~, ~~]
1                           [!]
2                            []
3                            []
4                       [., ^^]
                  ...          
908742                     [!!]
908743                      [.]
908744                   [., .]
908745    [!, ., ., ,, ,, ., ~]
908746                       []
Name: exclamation_mark, Length: 908747, dtype: object

In [6]:
def clean_text(text):
    text = text.replace(".", " ").strip()
    text = text.replace("·", " ").strip()
    pattern = '[^ ㄱ-ㅣ가-힣|0-9]+'
    text = re.sub(pattern=pattern, repl='', string=text) #한글, 숫자만 남김
    return text

# def clean_text(text):
#     text = text.replace(".", " ").strip()
#     text = text.replace("·", " ").strip()
#     pattern = '[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]'
#     text = re.sub(pattern=pattern, repl=' ', string=text) #한글, 숫자만 남김
#     words = text.split()
#     #print(words)
#     words = [word for word in words if not any(keyword in word for keyword in ['감자', '컬리', '마켓'])]
#     print(words)
#     #words = [word for word in words if ['감자','컬리', '마켓'] not in word]
#     clean_text = ' '.join(words)
#     return clean_text

In [7]:
def get_nouns(tokenizer, sentence):
    tagged = tokenizer.pos(sentence)
    #meaningful = [s for s, t in tagged if t in ['Noun', 'Verb', 'PreEomi', 'Unknown']]
    nouns = [s for s, t in tagged if t in ['NNG', 'NNP', 'VA', 'XR'] and len(s) >1]
    return nouns

def tokenize(df):
    tokenizer = Mecab() #오타 수정 기능도 있는 okt 선택. 
    processed_data = []
    for sent in tqdm(df['review_contents']):
        
        sentence = clean_text(str(sent).replace("\n", "").strip())
        #processed_data.append(get_meaningful(tokenizer, sentence))
        processed_data.append(get_nouns(tokenizer, sent))
    return processed_data

### [Python] 한국어 형태소 분석기 체험 및 비교(Okt, Mecab, Komoran, Kkma)
https://soohee410.github.io/compare_tagger

'재밓었다' 걍 다 뭉개짐. 띄어쓰기도 안 되고, 크리티컬한 오타가 존재할 때, 형태소 분석이 잘못될 가능성이 매우 높아지는 것 같다. Kkma와 Komoran은 더 구체적으로 형태소에 따라 단어를 쪼갬. 

Okt의 경우 stem=True, norm=True의 파라미터가 존재해서, 단어들을 알아서 조금 정규화해주고, 오타도 조금 수정해주는 기능이 있다. ‘사랑하고’ -> ‘사랑하다’로, ‘싶게’->싶다’로, ‘헤집어놓는’->‘헤집다’

kkma 너무 오래 걸림.

In [8]:
clean_text(' 걍 다 뭉개짐. 띄어쓰기도 안 되고, 크리티컬한 오타가 존재할 때, 형태소 분석이 잘못될 가능성이 매우 높아지는 것 같다. Kkma와 Komoran은 더 구체적으로 형태소에 따라 단어를 쪼갬.')

'걍 다 뭉개짐  띄어쓰기도 안 되고 크리티컬한 오타가 존재할 때 형태소 분석이 잘못될 가능성이 매우 높아지는 것 같다  와 은 더 구체적으로 형태소에 따라 단어를 쪼갬'

In [ ]:
def save_processed_data(processed_data):
    with open("_Mecabtokenized_data_"+file, 'w', newline="", encoding='utf-8') as f:
        writer = csv.writer(f)
        for data in processed_data:
            writer.writerow(data)
        
    
if __name__ == '__main__':
    #df = pd.read_csv(file, low_memory=False)
    df = data
    df.columns=['product_code', 'buyer_class', 'review_best_status', 'review_contents', 'review_letNum', 'pic_status', 'pics_num', 'helpful_num', 'review_reg_date', 'review_exp_date', 'product_name', 'product_name_letNum', 'brand', 'price', 'discount_price', 'discount_rate', 'product_phrase', 'coupon_status','coupon_discount_rate', 'product_characteristic', 'delivery_type', 'category', 'purchase_benefit', 'total_review_num', 'overwhelming_review_num', 'exclamation_mark']
    df.dropna(how='any')

    processed_data = tokenize(df)
    save_processed_data(processed_data)


  9%|███▏                              | 85791/908747 [00:21<01:42, 8063.68it/s]

In [ ]:
#file = 'comment_product_mapped.csv'
file = '과일견과쌀.csv'

In [ ]:
processed_data = [sent.strip().split(",") for sent in tqdm(open("_Mecabtokenized_data_"+file,'r',encoding='utf-8').readlines())]


In [ ]:
processed_data = DataFrame(processed_data)
processed_data[0] = processed_data[0].replace("", np.nan)
processed_data = processed_data[processed_data[0].notnull()]
processed_data = processed_data.values.tolist()

In [ ]:
processed_data2=[]
for i in tqdm(processed_data):
    i = list(filter(None, i))
    processed_data2.append(i)
processed_data = processed_data2
processed_data

In [ ]:
exclude = ['감자', '컬리', '마켓', '아요', '요조', '구매', '오이', '채소', '야채', '새우', '오징어', '딸기', '블루베리', '바나나', '아보카도', '귤', '머스켓','머스캣'] #구매?
processed2 = []
for sent in processed_data[:10]:
    temp_sent = []
    for word in sent:
        if word not in exclude:
            temp_sent.append(word)
    processed2.append(temp_sent)

processed2

In [ ]:
from gensim.models.ldamodel import LdaModel
from gensim.models.callbacks import CoherenceMetric
from gensim import corpora
from gensim.models.callbacks import PerplexityMetric

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
dictionary = corpora.Dictionary(processed_data)

In [ ]:
dictionary.filter_extremes(no_below=2, no_above=0.5)

#너무 흔한 단어나 너무 독특한 단어는 제외
#빈도가 1인 단어가 포함되면 너무 많은 단어가 생겨 분석에 혼선을 줄 수 있음
#따라서 저는 빈도가 2 이상인 단어와 전체의 50%로 이상 차지하는 단어는 필터링

In [ ]:
corpus = [dictionary.doc2bow(text) for text in tqdm(processed2)]

In [ ]:
processed2

In [ ]:
num_topics = 3
chunksize = 2000
passes = 20
iterations = 400
eval_every = None

temp = dictionary[0]
id2word = dictionary.id2token

In [ ]:
model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

bert topic & combined tm  중 더 잘 되는 거 고르기

평가기준. f1 x. 어떤 기준으로 뭐가 더 잘되는지 골라야하는가. 
도메인에 따라 어떤 가중치를 주는가.
다음에 찾아올 때는 피드백들 발전시킬 것. 

In [ ]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

In [ ]:
lda_visualization = gensimvis.prepare(model, corpus, dictionary, sort_topics=False)
pyLDAvis.save_html(lda_visualization, 'fruits_revised_topic3.html')

## 토픽별 element 출력해보기

In [ ]:
def replace_word(text):
    replaced_text = text.replace('요', '요 ')
    return replaced_text

In [ ]:
replace_word('최저가라서샀어요조아요')

채소 - 4,5 topic modeling -> 완전 동일한(겹치는) 뭉치들의 토픽들이 있다. 이걸 제외하고 생각한다면 잘 나눠진 것 같다. chaeso_revised_topics00.html